<a href="https://colab.research.google.com/github/DYLee87/UF_EV/blob/main/Training_YOLOV5_OCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0. Base Setting

In [1]:
# Google drive 연동

from google.colab import drive
drive.mount('/gdrive', force_remount = True)

Mounted at /gdrive


In [2]:
## Yolov5 git clone 및 requirements install

#%cd /gdrive/MyDrive/
#!git clone https://github.com/ultralytics/yolov5.git

%cd /
%cd /gdrive/MyDrive/yolov5/


#설치 후 runtime 다시 시작
#!pip install -r requirements.txt

/
/gdrive/MyDrive/yolov5


##1. Data preparation

In [3]:
## Google drive의 각 폴더에 raw img, label txt 파일을 준비
## 폴더의 파일명 및 수량은 반드시 동일해야함!!!!!!!!!!!!!
## ex ./xxxx/images/*.JPG
## ex ./xxxx/labels/*.JPG
## class file은 'images' & 'labels'의 상위 폴더로

from glob import glob
from sklearn.model_selection import train_test_split

f_path = '/gdrive/MyDrive/0.Image_Dataset/OCR_Test/0.training/'
f_path = '/gdrive/MyDrive/0.Image_Dataset/OCR_Test/'

img_list = glob(f_path + 'images/*.JPG')
img_list = sorted(img_list)
print(len(img_list))

txt_list = glob(f_path + 'labels/*.txt')

txt_list = sorted(txt_list)
img_list = [f_path + 'images/' + txt_list[i][48:56] + '.JPG' for i in range(len(txt_list)) ] # 임시 생성

print(len(img_list), " / ", len(txt_list))

train_img_list, val_img_list = train_test_split(img_list, test_size=0.2, random_state=2000)
print(len(train_img_list), len(val_img_list))


732
533  /  533
426 107


In [4]:
txt_list[0][48:56]

'IMG_3951'

In [5]:
img_list[0]

'/gdrive/MyDrive/0.Image_Dataset/OCR_Test/images/IMG_3951.JPG'

In [6]:
### Labeling Class 불러오기

file = open(f_path + 'classes.txt', "r+");
strings = file.readlines()
ocr_class = [word.strip('\n') for word in strings]
ocr_class[:5]

['PRODUCT NAME', 'WEEK Number', 'U.T.Q.G.S', 'LI/SS', 'E-MARK  &  NO']

In [7]:
## image 수량이 안맞을 경우 비교하기위한 func

img_li = list(map(lambda x : x[48:56], img_list))
txt_li = list(map(lambda x : x[48:56], txt_list))

for i in range(len(img_li)):
    if (img_li[i] == txt_li[i]) == False:
        print(i)
    elif i == (len(img_li)-1):
        print('end')

print(img_li[0:5])
print(txt_li[0:5])

end
['IMG_3951', 'IMG_3952', 'IMG_3954', 'IMG_3955', 'IMG_3956']
['IMG_3951', 'IMG_3952', 'IMG_3954', 'IMG_3955', 'IMG_3956']


##2. Data input 정보 생성(.yaml)
!!!! 반드시 첨부된 data.yaml 파일 사용

In [8]:
with open('train.txt', 'w') as f:
  f.write('\n'.join(train_img_list) + '\n')
  
with open('val.txt', 'w') as f:
  f.write('\n'.join(val_img_list) + '\n')

In [9]:
len(train_img_list[:5])

5

In [10]:
import yaml

with open('/gdrive/MyDrive/yolov5_dataset/data_custom.yaml', 'r') as f:
  try:
    data = yaml.load(f) #error시, full_load(f) or load(f)로 변경
  except:
    data = yaml.full_load(f)

print(data)

data['names'] = ocr_class
data['nc'] = len(ocr_class)
data['train'] = '/gdrive/MyDrive/yolov5/train.txt'
data['val'] = '/gdrive/MyDrive/yolov5/val.txt'

with open('data.yaml', 'w') as f:
  yaml.dump(data, f)

#print(data[])

{'train': '../train/images', 'val': '../valid/images', 'nc': 3, 'names': ['Tire', 'Tire_inner', 'Barcode']}


In [11]:
print(data['names'][:5])
print(data['nc'])
print(data['train'])

['PRODUCT NAME', 'WEEK Number', 'U.T.Q.G.S', 'LI/SS', 'E-MARK  &  NO']
113
/gdrive/MyDrive/yolov5/train.txt


##3. Training

 자세한 사항은 세부 항복은 train.py의 arg parser 확인
 
 --img : 이미지 처리 size, yolov5 github 참조
 
 --data : yolov5에 custom 된 yaml 파일
 
 --cfg : weights s에 대한 yaml 파일
 
 --weights : weights에 대한 부분은 yolov5 github 참조
 
 --name : train된 best.py가 저장될 folder 이름 ./runs/train/ 에 해당폴더이름으로 저장됨

In [ ]:
!python train.py --img 640 --batch 32 --epochs 2000 --data data.yaml --cfg ./models/yolov5m.yaml --weights ./weights/yolov5m.pt --name OCR_test_

train: weights=./weights/yolov5m.pt, cfg=./models/yolov5m.yaml, data=data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=2000, batch_size=32, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=OCR_test_, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-11-g63ddb6f torch 1.10.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, 

##4. Detecting

 자세한 사항은 세부 항복은 detect.py의 arg parser 확인
 
 --img : 이미지 처리 size, yolov5 github 참조
 
 --conf : bounding box할 treshold 값 (0~1 사이의 값)
 
 --source : detect할 image folder 또는 name
 
 해당폴더이름으로 저장됨

In [17]:
!python detect.py --weights runs/train/OCR_test_11/weights/best.pt --img 1280 --conf 0.25 --source '/gdrive/MyDrive/0.Image_Dataset/OCR_Test/DP_220224_P'

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.25, device='', exist_ok=False, hide_conf=False, hide_labels=False, img_size=640, iou_thres=0.45, line_thickness=3, name='exp', nosave=False, project='runs/detect', save_conf=False, save_crop=False, save_txt=False, source='/gdrive/MyDrive/0.Image_Dataset/OCR_Test/DP_220224_P', update=False, view_img=False, weights=['runs/train/OCR_test_11/weights/best.pt'])
YOLOv5 🚀 v5.0-64-g5189b3a torch 1.10.0+cu111 CPU

Fusing layers... 
/usr/local/lib/python3.7/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 224 layers, 7355974 parameters, 0 gradients, 17.3 GFLOPS
image 1/469 /gdrive/MyDrive/0.Image_Dataset/OCR_Test/DP_220224_P/IMG_5632.jpg: 640x480 1 SIZE, 1 MATERIAL, 1 DOT CODE

In [18]:
!python detect.py --weights runs/train/OCR_test_11/weights/best.pt --img 640 --conf 0.25 --source '/gdrive/MyDrive/0.Image_Dataset/OCR_Test/DP_220224_K'

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.25, device='', exist_ok=False, hide_conf=False, hide_labels=False, img_size=640, iou_thres=0.45, line_thickness=3, name='exp', nosave=False, project='runs/detect', save_conf=False, save_crop=False, save_txt=False, source='/gdrive/MyDrive/0.Image_Dataset/OCR_Test/DP_220224_K', update=False, view_img=False, weights=['runs/train/OCR_test_11/weights/best.pt'])
YOLOv5 🚀 v5.0-64-g5189b3a torch 1.10.0+cu111 CPU

Fusing layers... 
/usr/local/lib/python3.7/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 224 layers, 7355974 parameters, 0 gradients, 17.3 GFLOPS
image 1/409 /gdrive/MyDrive/0.Image_Dataset/OCR_Test/DP_220224_K/IMG_6550.jpg: 640x480 3 DOT CODEs, Done. (0.085s)
im

In [19]:
!python detect.py --weights runs/train/OCR_test_11/weights/best.pt --img 640 --conf 0.25 --source '/gdrive/MyDrive/0.Image_Dataset/OCR_Test/images_old'

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.25, device='', exist_ok=False, hide_conf=False, hide_labels=False, img_size=640, iou_thres=0.45, line_thickness=3, name='exp', nosave=False, project='runs/detect', save_conf=False, save_crop=False, save_txt=False, source='/gdrive/MyDrive/0.Image_Dataset/OCR_Test/images_old', update=False, view_img=False, weights=['runs/train/OCR_test_11/weights/best.pt'])
YOLOv5 🚀 v5.0-64-g5189b3a torch 1.10.0+cu111 CPU

Fusing layers... 
/usr/local/lib/python3.7/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 224 layers, 7355974 parameters, 0 gradients, 17.3 GFLOPS
image 1/820 /gdrive/MyDrive/0.Image_Dataset/OCR_Test/images_old/IMG_3951.JPG: 640x480 1 WEEK Number, 1 SIZE, 1 DOT COD